# Convolutional Neural Network Intro 
<center><img src='./images/cnn_example.PNG' width=800px></center> 

### Image Filter / Image Kernel 
- First, we do the convolution to the image to extract features. We create a filter/kernel (whcih uis basically a matrix) and apply convolution.
- In this kernel the values are designed by NN to extract features. NN will learn those and convolute with image.
- CNN is usually detect the edges (like sobel filter-right, left, top, bottom)
- In colour images we have 3 colour channels (RGB) with hights and widht of the image.
- After pooling we have fully connected layer.

<center><img src='./images/cnn_kernel.PNG' width=600px></center> 

Ref: Visualize the convolution: https://setosa.io/ev/image-kernels/

### Why to use CNN instead of ANN or FCNN?
- When the data becaomes very big, in ANN all neurons are fully connected. Its diffilcult to process data. However in CNN its not fully connected. it is locally connected.
- Once we extract feateures we do pooling tor reduce feature
- Once we do this process we than use fully connected layer after flattening.
- CNN is crunching the parameters down by doing convolution using filetering and further by pooling.

<center><img src='./images/cnn_local_connect.PNG' width=600px></center>  

### Pooling Layer
- Reduce the features further. It reduces the amount of data in an image by combining information from multiple vectors into fewer vectors
<center><img src='./images/pool_concept.jpg' width=600px></center>  

<center><img src='./images/pooling.PNG' width=600px></center>  

## Dataset: MNIST
- It has hand written numbers from 0 to 9.
- 60,000 images in the training set. 10,000 images for testing
- It has 28x28 pixels in an array. white = 0 , black =1.
- The images in the dataset are binarized, normalized, and centered to remove unnecessary noise and variations.
- Machine learning algorithms, such as neural networks or support vector machines, are used to classify the images into their respective categories (0-9).
- So images have border, whcih is gray scale and have values between 0 and 1.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Beforem we import data we whave to transform the data. We need tensor of 4 dimension in order to do things. 
# to keep track of number of images, height, widhtm and colour

# convert MNIST image files into tensor of 4-dimensions (# images, height, Width, Color)
transform = transforms.ToTensor()

# 1. Download MNIST Data

In [ ]:
# Pull the MNIST data save the training and test data
# Train Data
train_data = datasets.MNIST(root='cnn_data', train=True, download=True, transform=transform) # save locally so give any directory name
print(train_data)

# test data. just train=False
test_data = datasets.MNIST(root='cnn_data', train=False, download=True, transform=transform) 
print(test_data)

In [ ]:
# lets check where the data is 
pwd # remeber this to come back tho this directory
ls # you will not see the cnn data in the directory because it save data one directory behind

# -------------------------------------
cd ../
pwd
ls

cd cnn_data
ls

# cd ../ # Come back to original directory

# 2. Data Loader
2. Convolutional and Pooling Layers
lets look nitty grity of CNN.We want to upoad images in batches

In [ ]:
# Create batch of 10 images
train_loader = DataLoader(train_data, batch_size=10, shuffle =True)
test_loader = DataLoader(train_data, batch_size=10, shuffle =False)

# Understand the parts of CNN

In [ ]:
# Define CNN Model
# Describe convolutional layer and whats it doing (c convolutional layer)

# lets do it only for 1 image, 3 filters, 3 kernel size and stride =1. We can set padding to make size same
conv1 = nn.Conv2d(1, 6, 3, 1) # the output of this layer is 6 neurons, whcih we will pass as input to send con layer
conv2 = nn.Conv2d(6, 16, 3, 1)

# grab `1` MNIST image
for i, (X_Train, y_train) in enumerate(train_data):
    break

# Check the shape of image. 
print('X_Train Shape = ',X_Train.shape)

# convert image to 4D. 1 batch, 1 image, height, width
x = X_Train.view(1, 1, 28, 28)

#perform our first Convolution
x = F.relu(conv1(x)) 
print('Shape of x after Conv1 = ', x.shape)# con1 is applied to image. 
# >>torch.Size([1,6,26,26]) 1 image, 6 filters, after convolution the size is 26x26


In [ ]:
# Pass pooling layer of kernelsize =2, stride =2
x = F.max_pool2d(x,2,2)
print('Shape of x after pooling = ', x.shape)
# >>torch.Size([1,6,13,13]) 26/2 = 13

In [ ]:
# Do second conolutional layer and poling layer
x = F.relu(conv2(x)) 
print('Shape of x after Conv2 = ', x.shape)

# Apply poling
x = F.max_pool2d(x,2,2)
print('Shape of x after pooling = ', x.shape) # the size is round down instead of round up. ((28-2)/2 -2)/2


# Model setup to do all the studd automatically

In [ ]:
# Model class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.conv2D(1,6,3,1)
        self.conv2 = nn.conv2D(6,16,3,1)
        # Fully connected layer. 3 Fullly connected layer
        self.fc1 = nn.Linear(5*5&16, 120) # 120 arbitrary neurons
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # output is 10 or 10 classes. So make sure this is correct

    def forward(self, X):
        # First pass
        X = F.relu(conv1(X))
        X = F.max_pool2d(X,2,2) # 2x2 kernel and strid=2
        # second pass
        X = F.relu(conv2(X))
        X = F.max_pool2d(X,2,2)  # output of this is of size 16*5*5

        # Flatten the output
        X = X.view(-1, 16*5*5) # -1 so we can vary the batch size

        # Fully connected layer
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X) # No relu on the lst layer

        return F.log_softmax(X, dim=1)

In [ ]:
# Create an instance of model
torch.manual_seed(41)
model = ConvolutionalNetwork()

# Print model to check the architecture
model

In [ ]:
# Define loss and optimizer
criterion = nn.crossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 


# Train and Test CNN Model
https://www.youtube.com/watch?v=dGLPvNhjs4U&list=PLCC34OHNcOtpcgR9LEYSdi9r7XIbpkpK1&index=17

In [ ]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results
    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)


  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

In [ ]:
# Plotting the loss at epoch
train_losses = [tl.item() for tl in train_losses]  #First try without this and see error. convet tensor to python list
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validation Loss")
plt.title("Loss at Epoch")
plt.legend()

In [ ]:
# Plot the accuracy at the end of each epoch
plt.plot([t/600 for t in train_correct], label="Training Accuracy")
plt.plot([t/100 for t in test_correct], label="Validation Accuracy")
plt.title("Accuracy at the end of each Epoch")
plt.legend()

# Test the model

In [ ]:
# Load the test data
test_load_everything = DataLoader(test_data, batch_size=10000, shuffle=False) # load all the test images

with torch.no_grad():
  correct = 0
  for X_test, y_test in test_load_everything:
    y_val = model(X_test)
    predicted = torch.max(y_val, 1)[1]
    correct += (predicted == y_test).sum()

# Did for correct
print('Total Correct predictions out of',str(len(test_data)) + correct.item())
print('%age correct prediction', correct.item()/len(test_data)*100)
correct.item()/len(test_data)*100
# >> 9873 correct out of 10000 images

In [ ]:
# Grab an image: to check both image and label
test_data[4143] # Tensor with an image in it...at end, it shows the label

In [ ]:
# Grab just the data: we don't need the label
test_data[4143][0]

In [ ]:
# Reshape the data
test_data[4143][0].reshape(28,28)

# Show the image
plt.imshow(test_data[1978][0].reshape(28,28))

In [ ]:
# New Prediction the image with out trained model. Pass the image thru our model
model.eval()
with torch.no_grad():
  new_prediction = model(test_data[1978][0].view(1,1,28,28)) # batch size of 1, 1 color channel, 28x28 image

In [ ]:
# Check the new prediction...get probabilities
new_prediction

In [ ]:
new_prediction.argmax()